In [1]:
import os
import re
import pandas as pd
from pandas import Series,DataFrame
import numpy as np

In [2]:
alldata = []

In [3]:
# (using this as oppossed to native Pandas because txt doesn't injest easilly)
pdf_file = 'Texas.txt'
pdf = open(pdf_file, "r")

In [4]:
# puts the data from the pdf into a list, skipping new line characters
for line in pdf:
    alldata.append(line.strip())

In [5]:
alldata

['Schedule A',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'COMMITMENT FOR TITLE INSURANCE',
 '',
 'Issued By',
 '',
 'Chicago Title Insurance Company',
 '',
 'SCHEDULE A',
 '',
 '',
 'Effective Date:  October 6, 2015',
 '',
 'Issued: 10/28/2015',
 '',
 '1.',
 '',
 '',
 'The policy or policies to be issued are:',
 '',
 'GF No. FRE',
 '',
 '(a)',
 '',
 '',
 '(b)',
 '',
 '',
 '(c)',
 '',
 '(d)',
 '',
 '(e)',
 '',
 '(f)',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 "OWNER'S POLICY OF TITLE INSURANCE (Form T-1)",
 '(Not applicable for improved one-to-four family residential real estate)',
 'Policy Amount:',
 'PROPOSED INSURED:',
 "TEXAS RESIDENTIAL OWNER'S POLICY OF TITLE INSURANCE",
 '- ONE-TO-FOUR FAMILY RESIDENCES (Form T-1R)',
 'Policy Amount:',
 'PROPOSED INSURED:',
 'LOAN POLICY OF TITLE INSURANCE (Form T-2)',
 'Policy Amount:',
 'PROPOSED INSURED:',
 'Proposed Borrower:',
 'TEXAS SHORT FORM RESIDENTIAL LOAN POLICY OF TITLE INSURANCE (Form T-2R)',
 'Policy Amount:',
 'PROPOSED INSURED:',
 'Prop

In [6]:
# makes a data frame out of the data 
raw_pdf = pd.DataFrame(alldata)

In [7]:
# finds the rows with blank data and fills them with Nan so they can be dropped, and drops them
raw_pdf = raw_pdf[raw_pdf != '']
raw_pdf.dropna(inplace=True)

In [8]:
# lets reindex so that we don't have huge jumps in the index for dropped rows
raw_pdf.set_index(np.arange(0,len(raw_pdf)),inplace = True)

In [9]:
# creating the list of headings to hunt for, hard-cording from the pdf, and using pandas tricks to get the last line.
headings = ['SCHEDULE A','EXCEPTIONS FROM COVERAGE','SCHEDULE C', 'SCHEDULE D' ,(raw_pdf[0][raw_pdf.index[-1]])]

In [10]:
# create a dataframe to store the index locations for the starting line of each section
locations = [0]
df_locations = pd.DataFrame(locations)

In [11]:
# run each heading in our list through a search, and place those start index locations into a dataframe
for heading in headings:
    df_locations[heading] = raw_pdf[raw_pdf[0].str.match(heading)].index[0]

In [12]:
# some column labeling and cleanup, and sorting by order they appear in the pdf
df_locations.drop(0,axis=1,inplace=True)
df_locations = df_locations.rename({0:'start_line'}).transpose()
df_locations.sort_values(by='start_line',inplace=True)

In [13]:
# shifting to a numeric index so it is easy to iterate over rows in Dataframe
df_locations['sections'] = df_locations.index
df_locations.reset_index(drop=True, inplace=True)

In [14]:
# creating the column indicating the ending line for each section of the pdf
end = df_locations['start_line'][1:len(df_locations)].values
end = np.append(end,df_locations['start_line'][len(df_locations)-1:len(df_locations)].values)
df_locations['end_line'] = end

In [15]:
df_locations

,start_line,sections,end_line
0,4,SCHEDULE A,58
1,58,EXCEPTIONS FROM COVERAGE,152
2,152,SCHEDULE C,208
3,208,SCHEDULE D,262
4,262,Page 6,262


In [16]:
# now we can use "start_line" and "end_line" to define slices of the raw_pdf Dataframe and make section-specific chunks
# and output results to csv
for index, row in df_locations.iterrows():
    string = (pdf_file + "_" + "df" + "_" + str(row[1]))  
    print string
    string = raw_pdf[row[0]: row[2]]
    string.to_csv(path_or_buf=(pdf_file + "_" + "df" + "_" +str(row[1]) + ".csv"))

Texas.txt_df_SCHEDULE A
Texas.txt_df_EXCEPTIONS FROM COVERAGE
Texas.txt_df_SCHEDULE C
Texas.txt_df_SCHEDULE D
Texas.txt_df_Page 6
